In [1]:
#EfficientNet


In [2]:
from google.colab import drive
drive.mount("/content/drive")
!pwd
!ls /content/drive/MyDrive/Alzheimer_s\ Dataset.zip

Mounted at /content/drive
/content
'/content/drive/MyDrive/Alzheimer_s Dataset.zip'


In [3]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/Alzheimer_s Dataset.zip"

extracted_dir = "/content/Alzheimer_s_Dataset"  # Define the directory where files will be extracted

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Get the path to the inner directory containing the dataset files
inner_dir = os.path.join(extracted_dir, 'Alzheimer_s Dataset')

# Define paths to train data directory
train_dir = os.path.join(inner_dir, 'train')

# Check if the train directory exists
if not os.path.exists(train_dir):
    print("Train directory not found:", train_dir)
    exit()

# Define image size and batch size
img_size = (224, 224)
batch_size = 32  # You can adjust this based on your hardware and dataset size

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Additional rotation
    width_shift_range=0.2,  # Additional horizontal shift
    height_shift_range=0.2,  # Additional vertical shift
    fill_mode='nearest'  # Fill in missing pixels with the nearest available value
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Assuming images are organized in folders by class
)

# Load EfficientNetB0 model without top (classification) layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers with regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Dropout for regularization
    Dense(512, activation='relu'),  # Reduced number of neurons
    Dropout(0.5),  # Dropout for regularization
    Dense(256, activation='relu'),  # Further reduced number of neurons
    BatchNormalization(),  # Batch normalization layer
    Dense(4, activation='softmax')
])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fine-tune the last 6 layers
for layer in base_model.layers[:-6]:
    layer.trainable = False

# Re-compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model with fine-tuning
fine_tune_history = model.fit(train_generator, epochs=50, callbacks=[early_stopping])

# Save the fine-tuned model
model.save("/content/drive/MyDrive/fine_tuned_efficientnet_model.h5")


Found 5121 images belonging to 4 classes.
16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/50
161/161 [==============================] - ETA: 0s - loss: 1.9405 - accuracy: 0.2521

161/161 [==============================] - 83s 425ms/step - loss: 1.9405 - accuracy: 0.2521
Epoch 2/50
161/161 [==============================] - ETA: 0s - loss: 1.8662 - accuracy: 0.2533

161/161 [==============================] - 68s 425ms/step - loss: 1.8662 - accuracy: 0.2533
Epoch 3/50
161/161 [==============================] - ETA: 0s - loss: 1.8141 - accuracy: 0.2624

161/161 [==============================] - 67s 417ms/step - loss: 1.8141 - accuracy: 0.2624
Epoch 4/50
161/161 [==============================] - ETA: 0s - loss: 1.7952 - accuracy: 0.2638

161/161 [==============================] - 67s 416ms/step - loss: 1.7952 - accuracy: 0.2638
Epoch 5/50
161/161 [==============================] - ETA: 0s - loss: 1.7581 - accuracy: 0.2765

...

161/161 [==============================] - 69s 426ms/step - loss: 1.2414 - accuracy: 0.4286
Epoch 48/50
161/161 [==============================] - ETA: 0s - loss: 1.2248 - accuracy: 0.4400

161/161 [==============================] - 67s 417ms/step - loss: 1.2248 - accuracy: 0.4400
Epoch 49/50
161/161 [==============================] - ETA: 0s - loss: 1.2383 - accuracy: 0.4304

161/161 [==============================] - 66s 411ms/step - loss: 1.2383 - accuracy: 0.4304
Epoch 50/50
161/161 [==============================] - ETA: 0s - loss: 1.2341 - accuracy: 0.4314

161/161 [==============================] - 67s 414ms/step - loss: 1.2341 - accuracy: 0.4314


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import os
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
inner_dir = "/content/Alzheimer_s_Dataset/Alzheimer_s Dataset"
test_dir = os.path.join(inner_dir, 'test')

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Load test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/fine_tuned_efficientnet_model.h5")

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Generate predictions and classification report
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

print(classification_report(true_classes, predicted_classes, target_names=class_labels))


Found 1279 images belonging to 4 classes.
40/40 [==============================] - 7s 105ms/step - loss: 1.2383 - accuracy: 0.5004
Test Loss: 1.2382638454437256
Test Accuracy: 0.5003909468650818
40/40 [==============================] - 4s 64ms/step
                  precision    recall  f1-score   support

    MildDemented       0.00      0.00      0.00       179
ModerateDemented       0.00      0.00      0.00        12
     NonDemented       0.50      1.00      0.67       640
VeryMildDemented       0.00      0.00      0.00       448

        accuracy                           0.50      1279
       macro avg       0.13      0.25      0.17      1279
    weighted avg       0.25      0.50      0.33      1279



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
